In [1]:
import numpy as np
from scipy.integrate import solve_ivp

import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

# matplotlib imports
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation as animation

init_vprinting()

<img src="two_body_sketch.jpg" alt="Two Body Sketch" width=300>

We will attempt to predict the motion of two bodies in space, using the two-body problem. We will use the following assumptions:
* The two bodies are point masses
* The two bodies are not affected by any other bodies
* The two bodies are not affected by any other forces (e.g. friction, air resistance, etc.)
* The two bodies are not affected by relativity

To solve this, we could use Newton's laws of motions, but I would like to utilise Lagrangian formalism. This is because Lagrangian formalism is more general, and can be used to solve more complex problems. It is also more elegant, and is more intuitive to me.

We will use the following notation:

* $m_1$ and $m_2$ are the masses of the two bodies
* $x_1$ and $x_2$ are the positions of the two bodies
* $\dot{x}_1$ and $\dot{x}_2$ are the velocities of the two bodies
* $T$ is the kinetic energy of the system
* $V$ is the potential energy of the system
* $\mathcal{L}$ is the Lagrangian of the system

Our job is to find equations for the kinetic and potential energies of the system in terms of some general coordinates. These coordinates will be $x_1$ and $x_2$, the positions of our two bodies. Afterwards, we can get the Lagrangian of the system through the following equation:

$$
\mathcal{L} = T - V
$$

To get the kinetic energy, we can use the following equation:

$$
T = \frac{1}{2}m_1v_1^2 + \frac{1}{2}m_2v_2^2
$$

We need to find a way to express the velocities in terms of the positions. We know that the velocities are the time derivatives of the positions, so we can express the kinetic energy as follows:

$$
T = \frac{1}{2}m_1\dot{x}_1^2 + \frac{1}{2}m_2\dot{x}_2^2
$$

The Lagrangian is then:

$$
\mathcal{L} = \frac{1}{2}m_1\dot{x}_1^2 + \frac{1}{2}m_2\dot{x}_2^2 - V
$$

The problem is that we don't know what $V$ is, so we will have to derive it ourselves. If $x_1$ and $x_2$ are the position vectors of the two bodies with respect to a fixed origin, then we can write the relative position vector as follows:

$$
\vec{r} = x_1 - x_2
$$

Now we want to get an equation for the barycentre of the system. The barycentre is the point where the two bodies would balance if they were connected by a rigid rod. To get an equation for the barycentre in terms of our positions and the masses, we can use the following equation:

$$
\vec{r}_{b} = \frac{m_1x_1}{m_1 + m_2} + \frac{m_2x_2}{m_1 + m_2}
$$

We can use our relative position vector to get an equation for the barycentre in terms of the relative position and the masses:

$$
\begin{align}
\vec{r}_{b} &= \frac{m_1x_1}{m_1 + m_2} + \frac{m_2x_2}{m_1 + m_2} \\
&= \frac{m_1(x_2 + \vec{r})}{m_1 + m_2} + \frac{m_2x_2}{m_1 + m_2} \\
&= \frac{m_1x_2 + m_1\vec{r}}{m_1 + m_2} + \frac{m_2x_2}{m_1 + m_2} \\
&= \frac{m_1x_2 + m_1\vec{r} + m_2x_2}{m_1 + m_2} \\
&= \frac{m_1x_2 + m_2x_2 + m_1\vec{r}}{m_1 + m_2} \\
&= \frac{(m_1 + m_2)x_2 + m_1\vec{r}}{m_1 + m_2} \\
&= x_2 + \frac{m_1\vec{r}}{m_1 + m_2} \\
\end{align}
$$

In terms of $x_1$, it is:

$$
\vec{r}_{b} = x_1 - \frac{m_2\vec{r}}{m_1 + m_2}
$$

Therefore, the derivatives of the barycentre are:

$$
\begin{align}
\dot{\vec{r}}_{b} &= \dot{x}_1 - \frac{m_2\dot{\vec{r}}}{m_1 + m_2} = \dot{x}_2 + \frac{m_1\dot{\vec{r}}}{m_1 + m_2} \\
\ddot{\vec{r}}_{b} &= \ddot{x}_1 - \frac{m_2\ddot{\vec{r}}}{m_1 + m_2} = \ddot{x}_2 + \frac{m_1\ddot{\vec{r}}}{m_1 + m_2} \\
\end{align}
$$

If we define $\mu$ as the reduced mass of the system ($\mu = \frac{m_1m_2}{m_1 + m_2}$), then we can write the kinetic energy of the system as follows:

$$
\begin{align}
T &= \frac{1}{2}m_1\dot{x}_1^2 + \frac{1}{2}m_2\dot{x}_2^2 \\
&= \frac{1}{2}m_1\left(\dot{\vec{r}}_{b} + \frac{m_2\dot{\vec{r}}}{m_1 + m_2}\right)^2 + \frac{1}{2}m_2\left(\dot{\vec{r}}_{b} - \frac{m_1\dot{\vec{r}}}{m_1 + m_2}\right)^2 \\
&= \frac{1}{2}m_1\left(\dot{\vec{r}}_{b}^2 + \frac{2m_2\dot{\vec{r}}_{b}\dot{\vec{r}}}{m_1 + m_2} + \frac{m_2^2\dot{\vec{r}}^2}{(m_1 + m_2)^2}\right) + \frac{1}{2}m_2\left(\dot{\vec{r}}_{b}^2 - \frac{2m_1\dot{\vec{r}}_{b}\dot{\vec{r}}}{m_1 + m_2} + \frac{m_1^2\dot{\vec{r}}^2}{(m_1 + m_2)^2}\right) \\
&= \frac{1}{2}m_1\dot{\vec{r}}_{b}^2 + \frac{1}{2}m_2\dot{\vec{r}}_{b}^2 + \frac{m_1m_2\dot{\vec{r}}_{b}\dot{\vec{r}}}{m_1 + m_2} - \frac{m_1m_2\dot{\vec{r}}_{b}\dot{\vec{r}}}{m_1 + m_2} + \frac{m_1m_2^2\dot{\vec{r}}^2}{2(m_1 + m_2)^2} + \frac{m_1^2m_2\dot{\vec{r}}^2}{2(m_1 + m_2)^2} \\
&= \frac{1}{2}m_1\dot{\vec{r}}_{b}^2 + \frac{1}{2}m_2\dot{\vec{r}}_{b}^2 + \frac{m_1m_2^2\dot{\vec{r}}^2 + m_1^2m_2\dot{\vec{r}}^2}{2(m_1 + m_2)^2} \\
&= \frac{1}{2}\dot{\vec{r}}_{b}^2\left(m_1 + m_2\right) + \frac{m_1m_2\dot{\vec{r}}^2}{2(m_1 + m_2)} \\
&= \frac{1}{2}\dot{\vec{r}}_{b}^2\left(m_1 + m_2\right) + \frac{\mu\dot{\vec{r}}^2}{2} \\
\end{align}
$$